In [ ]:
# Connect notebook to your google drive (run this everytime)
import os, sys
from google.colab import drive
ROOT = "/content/drive/MyDrive/Colab Notebooks/"
CLASS_PATH = ROOT+ "CSci5511/"

print ('CLASS_PATH=',CLASS_PATH)
!mkdir "{CLASS_PATH}"
%cd "{CLASS_PATH}"
%cd "{CLASS_PATH}"

# Download the AIMA python code to your drive (only do this once ever):
!git clone https://github.com/aimacode/aima-python.git

# verify that you see the package folder
!pwd
!ls -al "{CLASS_PATH}"

# # Install the AIMA package
# CLASS_PATH = os.path.join(ROOT, "CSci5511/")
# CLASS_PATH = ROOT+ "CSci5511/"
# AIMA_PATH = os.path.join(CLASS_PATH, "aima-python")
# AIMA_PATH=CLASS_PATH+"aima-python"
# %cd "{AIMA_PATH}"
# Do only once the first time you use the AIMA package!
# !pip install -r requirements.txt
# !git submodule init
# !git submodule update

# (optional, uncomment to run) Test that the installation worked:
#!pip install pytest
#!py.test

CLASS_PATH= /content/drive/MyDrive/Colab Notebooks/CSci5511/
mkdir: cannot create directory ‘/content/drive/MyDrive/Colab Notebooks/CSci5511/’: No such file or directory
[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/CSci5511/'
/content
[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/CSci5511/'
/content
Cloning into 'aima-python'...
remote: Enumerating objects: 5095, done.
remote: Total 5095 (delta 0), reused 0 (delta 0), pack-reused 5095 (from 1)
Receiving objects: 100% (5095/5095), 17.44 MiB | 17.05 MiB/s, done.
Resolving deltas: 100% (3418/3418), done.
/content
ls: cannot access '/content/drive/MyDrive/Colab Notebooks/CSci5511/': No such file or directory


In [ ]:
AIMA_PATH = "/content/aima-python"
%cd "{AIMA_PATH}"

/content/aima-python


In [ ]:
def alpha_beta_cutoff_search_custom_evaluate(state, game, depth=6):
    """Search game to determine best action using alpha-beta pruning.
    If depth is not provided, it evaluates the entire game tree."""

    def max_value(state, alpha, beta, current_depth):
        if game.terminal_test(state) or (depth is not None and current_depth == 0):
            return custom_evaluate(state, game)
        v = float('-inf')
        for action in game.actions(state):
            v = max(v, min_value(game.result(state, action), alpha, beta, current_depth - 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, current_depth):
        if game.terminal_test(state) or (depth is not None and current_depth == 0):
            return custom_evaluate(state, game)
        v = float('inf')
        for action in game.actions(state):
            v = min(v, max_value(game.result(state, action), alpha, beta, current_depth - 1))
            # if v <= alpha:
            if alpha > v:
                return v
            beta = min(beta, v)
        return v

    # Alpha-beta search starts here
    best_score = float('-inf')
    beta = float('inf')
    best_action = None
    for action in game.actions(state):
        v = min_value(game.result(state, action), best_score, beta, (depth if depth else float('inf')) - 1)
        if v > best_score:
            best_score = v
            best_action = action
    return best_action

In [ ]:
def alpha_beta_cutoff_search(state, game, depth=5):
    """Search game to determine best action using alpha-beta pruning.
    If depth is not provided, it evaluates the entire game tree."""

    def max_value(state, alpha, beta, current_depth):
        if game.terminal_test(state) or (depth is not None and current_depth == 0):
            return evaluate(state, game)
        v = float('-inf')
        for action in game.actions(state):
            v = max(v, min_value(game.result(state, action), alpha, beta, current_depth - 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, current_depth):
        if game.terminal_test(state) or (depth is not None and current_depth == 0):
            return evaluate(state, game)
        v = float('inf')
        for action in game.actions(state):
            v = min(v, max_value(game.result(state, action), alpha, beta, current_depth - 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Alpha-beta search starts here
    best_score = float('-inf')
    beta = float('inf')
    best_action = None
    for action in game.actions(state):
        v = min_value(game.result(state, action), best_score, beta, (depth if depth else float('inf')) - 1)
        if v > best_score:
            best_score = v
            best_action = action
    return best_action


def evaluate(state, game):
    """Heuristic evaluation of a game state for Connect 4."""
    player = game.to_move(state)
    opponent = 'O' if player == 'X' else 'X'

    def count_sequences(state, token, length):
        """Count sequences of a given length for the specified token."""
        count = 0
        for row in state.board:
            count += check_line(row, token, length)
        for col in zip(*state.board):  # Check columns
            count += check_line(col, token, length)
        for diag in diagonals(state.board):  # Check diagonals
            count += check_line(diag, token, length)
        return count

    def check_line(line, token, length):
        """Count unbroken sequences of 'length' for 'token' in a line."""
        return sum(1 for i in range(len(line) - length + 1)
                   if all(x == token for x in line[i:i + length]))

    def diagonals(matrix):
        """Get all diagonals in a matrix."""
        diags = []
        for p in range(-len(matrix) + 1, len(matrix[0])):
            diags.append([matrix[i][i - p] for i in range(len(matrix)) if 0 <= i - p < len(matrix[0])])
        return diags

    # Heuristic calculation
    score = 0
    score += count_sequences(state, player, 4) * 1000  # Winning line
    score += count_sequences(state, player, 3) * 10  # Threatening line
    score += count_sequences(state, player, 2) * 1  # Build-up line
    score -= count_sequences(state, opponent, 4) * 1000  # Opponent win
    score -= count_sequences(state, opponent, 3) * 10  # Opponent threat
    return score

# state is board,
evaluationTable = [[3, 4, 5, 7, 5, 4, 3],
                                [ 4, 6, 8, 10, 8, 6, 4],
                                [ 5, 8, 11, 13, 11, 8, 5],
                                [ 5, 8, 11, 13, 11, 8, 5],
                                [ 4, 6, 8, 10, 8, 6, 4],
                                [ 3, 4, 5, 7, 5, 4, 3]]

def custom_evaluate(state, game):
        # Init evaluate function
        # player2 = "X" if player1 == "O" else "O"
        player1 = game.to_move(state)
        player2 = "X" if player1 == "O" else "O"
        # 138 * 2 = Entire block's sum in the evalutaion table
        utility = 300

        def count_sequences(state, token, length):
          """Count sequences of a given length for the specified token."""
          count = 0
          for row in state.board:
              count += check_line(row, token, length)
          for col in zip(*state.board):  # Check columns
              count += check_line(col, token, length)
          for diag in diagonals(state.board):  # Check diagonals
              count += check_line(diag, token, length)
          return count

        def check_line(line, token, length):
          """Count unbroken sequences of 'length' for 'token' in a line."""
          return sum(1 for i in range(len(line) - length + 1)
                    if all(x == token for x in line[i:i + length]))

        def diagonals(matrix):
          """Get all diagonals in a matrix."""
          diags = []
          for p in range(-len(matrix) + 1, len(matrix[0])):
              diags.append([matrix[i][i - p] for i in range(len(matrix)) if 0 <= i - p < len(matrix[0])])
          return diags

        # Check first if connections with size 4 exist
        if count_sequences(state, player1, 4) > 0:
            if count_sequences(state, player2, 4) > 0:
                return 0
            return 2 * utility
        if count_sequences(state, player2, 4) > 0:
            return -2 * utility

        # tmp_board = board.get()
        tmp_state = state.board
        # If not, use heuristic
        sum_u = 0
        for i in [0,1,2,3,4,5]:
            for j in [0,1,2,3,4,5,6]:
                if (tmp_state[i][j] == player1):
                    sum_u += evaluationTable[i][j]
                else:
                    if (tmp_state[i][j] == player2):
                        sum_u -= evaluationTable[i][j]

        # 내가 돌을 놓은 다음 상황에서,
        # 1) 내 돌이 3개 연속인 경우 * 3 의 Weight 를 준다
        # 2) 상대방의 돌이 2개 연속인 경우 * 3 의 - Weight 를 준다. (상대방이 돌을 놓아 3개가 만들어질 경우에 대해 미리 방지)
        # 3) 내 돌이 2개 연속인 경우 * 1 의 Weight 를 준다.
        sum_u += count_sequences(state, player1, 3) * 3 + count_sequences(state, player1, 2) * 1
        sum_u -= count_sequences(state, player2, 2) * 2

        return sum_u

def negamax(state, game, depth, alpha=float('-inf'), beta=float('inf'), color=1):
    """
    Negamax algorithm with alpha-beta pruning.

    Parameters:
        state: Current game state.
        game: The game object with defined methods (actions, result, terminal_test, utility).
        depth: Maximum depth to search.
        alpha: Alpha value for pruning.
        beta: Beta value for pruning.
        color: 1 for maximizing player, -1 for minimizing player.

    Returns:
        (value, action): Best value and corresponding action for the current player.
    """
    if game.terminal_test(state) or depth == 0:
        return color * evaluate(state, game), None

    best_value = float('-inf')
    best_action = None

    for action in game.actions(state):
        next_state = game.result(state, action)
        value, _ = negamax(next_state, game, depth - 1, -beta, -alpha, -color)
        value = -value  # Negate the result because we're switching perspectives

        if value > best_value:
            best_value = value
            best_action = action

        alpha = max(alpha, value)
        if alpha >= beta:
            break  # Beta cutoff

    return best_value, best_action




In [ ]:
from games import *
from games4e import *

# /content/aima-python

class ConnectFour(Game):
    """Connect Four implementation."""

    def __init__(self, rows=6, cols=7, k=4):
        self.rows = rows
        self.cols = cols
        self.k = k
        self.initial = GameState(
            to_move='X', utility=0, board=[['.' for _ in range(cols)] for _ in range(rows)],
            moves=list(range(cols))
        )

    def actions(self, state):
        """Return the list of valid columns where a piece can be dropped."""
        return [col for col in state.moves if state.board[0][col] == '.']

    def result(self, state, move):
        """Return the new state after a move."""
        board = [row[:] for row in state.board]
        for row in range(self.rows - 1, -1, -1):  # Find the lowest empty spot
            if board[row][move] == '.':
                board[row][move] = state.to_move
                break

        utility = self.compute_utility(board, move, state.to_move)
        to_move = 'O' if state.to_move == 'X' else 'X'
        moves = self.actions(GameState(to_move, utility, board, state.moves))
        return GameState(to_move, utility, board, moves)

    def utility(self, state, player):
        """Return the utility value of the state for a player."""
        return state.utility if player == 'X' else -state.utility

    def terminal_test(self, state):
        """Check if the game is over."""
        return state.utility != 0 or all(col != '.' for col in state.board[0])

    def display(self, state):
        """Print the board."""
        for row in state.board:
            print(' '.join(row))
        print('-' * self.cols * 2)

    def compute_utility(self, board, move, player):
        """Compute utility: +1 if player wins, -1 if opponent wins, else 0."""
        for dx, dy in [(0, 1), (1, 0), (1, 1), (1, -1)]:
            if self.k_in_row(board, player, dx, dy):
                return +1 if player == 'X' else -1
        return 0

    def k_in_row(self, board, player, dx, dy):
        """Check if there are k consecutive pieces in a row for player."""
        for x in range(self.rows):
            for y in range(self.cols):
                if all(
                    0 <= x + i * dx < self.rows and
                    0 <= y + i * dy < self.cols and
                    board[x + i * dx][y + i * dy] == player
                    for i in range(self.k)
                ):
                    return True
        return False


# Game setup and execution
def play_connect_four():
    game = ConnectFour()
    state = game.initial

    def human_player(game, state):
        game.display(state)
        move = int(input("Your move (0-6): "))
        return move

    while True:
        print("\nPlayer 1:")
        state = game.result(state, alpha_beta_cutoff_search_custom_evaluate(state, game))
        # state = game.result(state, alpha_beta_cutoff_search(state, game))
        # state = game.result(state, mcts_player(game,state))
        game.display(state)
        if game.terminal_test(state):
            break
        #use this for everything other than negamax
        # print("\nPlayer 2:")
        # state = game.result(state, mcts_player(game,state))
        # # state = game.result(state, alpha_beta_cutoff_search_custom_evaluate(state, game))
        # # state = game.result(state, alpha_beta_cutoff_search(state, game))
        # game.display(state)
        # if game.terminal_test(state):
        #     break

        # use this for negamax
        print("\nPlayer 2 (AI - Negamax):")
        _, action = negamax(state, game, depth=5)
        print(f"AI chooses column {action}")
        state = game.result(state, action)
        game.display(state)
        if game.terminal_test(state):
            break

        # minimax thinge without alpha beta pruning
        # print("\nPlayer 2 (AI - Minimax):")
        # state = game.result(state, minmax_decision(state, game))
        # game.display(state)
        # if game.terminal_test(state):
        #     break

    #game.display(state)
    if state.utility == 1:
        print("Player 1 (X) wins!")
        return 1
    elif state.utility == -1:
        print("Player 2 (O) wins!")
        return -1
    else:
        print("It's a tie!")
        return 0


In [ ]:
# no need to run anymore

# import time

# start = time.time()

# print(play_connect_four())
# end = time.time()

# print(end - start)

In [ ]:
import time

p1_win = 0
p2_win = 0
tie = 0
winner = 0
total_time = 0
for i in range(50):
  print("--------------------------",i)
  start = time.time()
  winner = play_connect_four()
  end = time.time()
  total_time = total_time + (end - start)
  if winner == 0:
    tie = tie + 1
  elif winner == 1:
    p1_win = p1_win + 1
  elif winner == -1:
    p2_win = p2_win + 1


print(total_time)
print(p1_win)
print(p2_win)
print(tie)

Streaming output truncated to the last 5000 lines.
Player 2 (AI - Negamax):
AI chooses column 3
. . . O . . .
. . . X . . .
. . . O . . .
. . . X . . .
O X . X . . .
O O . X . . .
--------------

Player 1:
. . . O . . .
. . . X . . .
. . . O . . .
X . . X . . .
O X . X . . .
O O . X . . .
--------------

Player 2 (AI - Negamax):
AI chooses column 4
. . . O . . .
. . . X . . .
. . . O . . .
X . . X . . .
O X . X . . .
O O . X O . .
--------------

Player 1:
. . . O . . .
. . . X . . .
. . . O . . .
X . . X . . .
O X . X . . .
O O X X O . .
--------------

Player 2 (AI - Negamax):
AI chooses column 2
. . . O . . .
. . . X . . .
. . . O . . .
X . . X . . .
O X O X . . .
O O X X O . .
--------------

Player 1:
. . . O . . .
. . . X . . .
. . . O . . .
X . X X . . .
O X O X . . .
O O X X O . .
--------------

Player 2 (AI - Negamax):
AI chooses column 1
. . . O . . .
. . . X . . .
. . . O . . .
X O X X . . .
O X O X . . .
O O X X O . .
--------------

Player 1:
. . . O . . .
. . . X . . .
.

In [ ]:
print(i)

49


In [ ]:
print(total_time)
print(p1_win)
print(p2_win)
print(tie)

2324.4496445655823
50
0
0
